# Predict Tg: Cross Validation
This notebook provides a demo of using the PolymerLearn package with PolymerGNN to predict Tg values from the dataset

In [1]:
import os
import torch
import pandas as pd
from polymerlearn.utils import get_IV_add, GraphDataset
from polymerlearn.utils.train_graphs import get_IV_add_nolog

# Load data from local path:
data = pd.read_csv(os.path.join('../dataset', # May need to change path, depending on location of data 
            'pub_data.csv'))

add = get_IV_add_nolog(data) # Use same values added for IV

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dataset = GraphDataset(
    data = data,
    structure_dir = '../Structures/AG/xyz',
    Y_target=['IV', 'Tg'],
    test_size = 0.2,
    add_features=add,#torch.from_numpy(add).to(device),
    standard_scale = True,
    device = device
)

/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:437: RuntimeWarning: invalid value encountered in greater
  acid_hit = (data.iloc[i,ac[0]:ac[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:441: RuntimeWarning: invalid value encountered in greater
  glycol_hit = (data.iloc[i,gc[0]:gc[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:437: RuntimeWarning: invalid value encountered in greater
  acid_hit = (data.iloc[i,ac[0]:ac[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:441: RuntimeWarning: invalid value encountered in greater
  glycol_hit = (data.iloc[i,gc[0]:gc[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:437: RuntimeWarning: invalid value encountered in greater
  acid_hit = (data.iloc[i,ac[0]:ac[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:441: RuntimeWarning: invalid value encountered in greater
  glycol_hit = (data.iloc[i,gc[

/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:437: RuntimeWarning: invalid value encountered in greater
  acid_hit = (data.iloc[i,ac[0]:ac[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:441: RuntimeWarning: invalid value encountered in greater
  glycol_hit = (data.iloc[i,gc[0]:gc[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:437: RuntimeWarning: invalid value encountered in greater
  acid_hit = (data.iloc[i,ac[0]:ac[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:441: RuntimeWarning: invalid value encountered in greater
  glycol_hit = (data.iloc[i,gc[0]:gc[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:437: RuntimeWarning: invalid value encountered in greater
  acid_hit = (data.iloc[i,ac[0]:ac[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:441: RuntimeWarning: invalid value encountered in greater
  glycol_hit = (data.iloc[i,gc[

/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:437: RuntimeWarning: invalid value encountered in greater
  acid_hit = (data.iloc[i,ac[0]:ac[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:441: RuntimeWarning: invalid value encountered in greater
  glycol_hit = (data.iloc[i,gc[0]:gc[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:437: RuntimeWarning: invalid value encountered in greater
  acid_hit = (data.iloc[i,ac[0]:ac[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:441: RuntimeWarning: invalid value encountered in greater
  glycol_hit = (data.iloc[i,gc[0]:gc[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:437: RuntimeWarning: invalid value encountered in greater
  acid_hit = (data.iloc[i,ac[0]:ac[1]].to_numpy() > 0)
/home/owq978/PolymerGNN/polymerlearn/utils/graph_prep.py:441: RuntimeWarning: invalid value encountered in greater
  glycol_hit = (data.iloc[i,gc[

Now we'll instantiate our model, using the standard IV model that gets the best prediction scores.

In [2]:
from polymerlearn.models.gnn import PolymerGNN_Joint_test
from polymerlearn.utils import CV_eval_joint

model_generator_kwargs = {
    'input_feat': 6,         # How many input features on each node; don't change this
    'hidden_channels': 32,   # How many intermediate dimensions to use in model
                            # Can change this ^^
    'num_additional': 4      # How many additional resin properties to include in the prediction
                            # Corresponds to the number in get_IV_add
}

optimizer_gen = torch.optim.AdamW
criterion = torch.nn.MSELoss()

all_predictions, all_y, all_inds = CV_eval_joint(
    dataset = dataset,
    model_generator = PolymerGNN_Joint_test,
    optimizer_generator = optimizer_gen,
    criterion = criterion,
    model_generator_kwargs = model_generator_kwargs,
    optimizer_kwargs = {'lr': 0.0001, 'weight_decay':0.01},
    epochs = 200,
    batch_size = 64,
    verbose = 1,
    device = device,
    gamma = 1e5
)

Fold: 1 : 0, Train r2 IV, Tg: -0.4123, -1.0226 	 Train Loss: 1949771.8834
Fold: 1 : 50, Train r2 IV, Tg: 0.9910, 0.7441 	 Train Loss: 106231.6160
Fold: 1 : 100, Train r2 IV, Tg: 0.9876, 0.9820 	 Train Loss: 76217.6825
Fold: 1 : 150, Train r2 IV, Tg: 0.9996, 0.8777 	 Train Loss: 43798.9349
Fold: 1 : 200, Train r2 IV, Tg: 0.0903, 0.9888 	 Train Loss: 56584.3289
Fold: 1 	 Test r2: 0.7294 	 r2_IV: 0.6562 	 r2_Tg: 0.8027 	 MSE: 133.6221 	 MSE_IV: 0.0299 	 MSE_Tg: 267.2144 	 MAE: 6.2618 	 MAE_IV: 0.0965 	 MAE_Tg: 12.4271
Fold: 2 : 0, Train r2 IV, Tg: -1.0354, -0.9127 	 Train Loss: 851837.3219


KeyboardInterrupt: 

Note that the cross validation, while more stable than a simple train/test split, is relatively unstable, meaning you could get a wide range of scores for any given run.

Now we can plot the model predictions vs. errors:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Parse the output from the CV:
Y = np.array(all_y)
Y_iv = Y[:,0]
Y_tg = Y[:,1]

pred = np.array(all_predictions)
pred_iv = pred[:,0]
pred_tg = pred[:,1]

plt.figure(dpi=150)
plt.plot(Y_iv, Y_iv, marker = 's', c = 'black', label = 'Actual IV')
plt.scatter(Y_iv, pred_iv, c = 'green', label = 'Predicted IV')
plt.xlabel('Actual IV')
plt.ylabel('Predicted IV')
plt.title('Predicted IV on Test Set (Joint model)')
plt.legend()
plt.show()

In [ ]:
plt.figure(dpi=150)
plt.plot(Y_tg, Y_tg, marker = 's', c = 'black', label = 'Actual Tg')
plt.scatter(Y_tg, pred_tg, c = 'red', label = 'Predicted Tg')
plt.xlabel('Actual Tg')
plt.ylabel('Predicted Tg')
plt.title('Predicted Tg on Test Set (Joint model)')
plt.legend()
plt.show()